In [1]:
# import libraries and define important variables

import logging
import os
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, Dense, Flatten, Dropout, RandomZoom, RandomContrast, Reshape, multiply
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation, RandomTranslation
from tensorflow.keras.optimizers import RMSprop, Adamax
from tensorflow.keras.metrics import CategoricalAccuracy, sparse_categorical_accuracy
from tensorflow.keras.callbacks import ModelCheckpoint

BATCH_SIZE = 32
IMG_SIZE = 299
SEED = 2021
VAL_SPLIT = 0.2
EPOCHS = 10
AUTOTUNE = tf.data.AUTOTUNE
LEARNING_RATE = 1e-5
file_dir = os.getcwd()
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

In [2]:

def data_augmentation(dataset, SEED):
    
    ## use data augmentation to artificially introduce sample diversity in training data
    
    rotation_augment = Sequential([RandomRotation(0.5)])
    translation_augment = Sequential([RandomTranslation(height_factor=0.15, width_factor=0.15, fill_mode='reflect')])
    flip_augment = Sequential([RandomFlip('horizontal_and_vertical')])
    zoom_augment = Sequential([RandomZoom(height_factor=0.3, fill_mode='reflect')])
    contrast_augment = Sequential([RandomContrast(factor=0.6)])
    
    dataset_1 = dataset.map(lambda x, y: (rotation_augment(x), y))
    dataset_2 = dataset.map(lambda x, y: (translation_augment(x), y))
    dataset_3 = dataset.map(lambda x, y: (flip_augment(x), y))
    dataset_4 = dataset.map(lambda x, y: (zoom_augment(x), y))
    dataset_5 = dataset.map(lambda x, y: (contrast_augment(x), y))
    
    dataset = dataset.concatenate(dataset_1)
    dataset = dataset.concatenate(dataset_2)
    dataset = dataset.concatenate(dataset_3)
    dataset = dataset.concatenate(dataset_4)
    dataset = dataset.concatenate(dataset_5)
    
    return dataset

def png_to_tfds(file_dir, BATCH_SIZE, IMG_SIZE, SEED, VAL_SPLIT, AUTOTUNE, model_name):
    
    ## preprocessing of images
    
    train_dir = file_dir + '\\train'
    test_dir = file_dir + '\\test'
    
    train_dataset = tf.keras.utils.image_dataset_from_directory(
        train_dir, labels='inferred', label_mode='categorical', class_names=None,
        color_mode='rgb', batch_size=BATCH_SIZE, image_size=(IMG_SIZE, IMG_SIZE),
        shuffle=True, seed=SEED, validation_split=VAL_SPLIT, subset='training',
        interpolation='bilinear', follow_links=False, crop_to_aspect_ratio=False
    )
    
    val_dataset = tf.keras.utils.image_dataset_from_directory(
        train_dir, labels='inferred', label_mode='categorical', class_names=None,
        color_mode='rgb', batch_size=BATCH_SIZE, image_size=(IMG_SIZE, IMG_SIZE),
        shuffle=True, seed=SEED, validation_split=VAL_SPLIT, subset='validation',
        interpolation='bilinear', follow_links=False, crop_to_aspect_ratio=False
    )
    
    test_dataset = tf.keras.utils.image_dataset_from_directory(
        test_dir, labels=None, label_mode=None, class_names=None,
        color_mode='rgb', batch_size=1, image_size=(IMG_SIZE, IMG_SIZE),
        shuffle=False, seed=None, validation_split=None, subset=None,
        interpolation='bilinear', follow_links=False, crop_to_aspect_ratio=False
    )
    
    ## normalization of pixels to (0, 1) / not required if the base model is resnetv2 or inception_resnet_v2
    ## resnetv2 & inception_resnet_v2 requires a specific preprocess input to normalize pixels to (-1, 1)
    
    if model_name == 'resnetv2':
        train_dataset = train_dataset.map(lambda x, y: (tf.keras.applications.resnet_v2.preprocess_input(x), y))
        val_dataset = val_dataset.map(lambda x, y: (tf.keras.applications.resnet_v2.preprocess_input(x), y))
        test_dataset = test_dataset.map(lambda x: (tf.keras.applications.resnet_v2.preprocess_input(x)))
    elif model_name == 'inception_resnet_v2':
        train_dataset = train_dataset.map(lambda x, y: (tf.keras.applications.inception_resnet_v2.preprocess_input(x), y))
        val_dataset = val_dataset.map(lambda x, y: (tf.keras.applications.inception_resnet_v2.preprocess_input(x), y))
        test_dataset = test_dataset.map(lambda x: (tf.keras.applications.inception_resnet_v2.preprocess_input(x)))

    else:
        normalization_layer = tf.keras.layers.Rescaling(1./255)
        train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
        val_dataset = val_dataset.map(lambda x, y: (normalization_layer(x), y))
        test_dataset = test_dataset.map(lambda x: (normalization_layer(x)))
    
    ## use data augmentation to artificially introduce sample diversity in training data
    
    train_dataset = data_augmentation(train_dataset, SEED)
    val_dataset = data_augmentation(val_dataset, SEED)
    
    ## for efficiency-ish (...) in the pipeline performance
    
    train_dataset = train_dataset.cache().prefetch(AUTOTUNE)
    val_dataset = val_dataset.cache().prefetch(AUTOTUNE)
    test_dataset = test_dataset.cache().prefetch(AUTOTUNE)
    
    print('\n')
    print('Size of training dataset is: ', len(train_dataset)*BATCH_SIZE)
    print('Size of validation dataset is: ', len(val_dataset)*BATCH_SIZE)
    print('Size of testing dataset is: ', len(test_dataset))
    print('\n')
    
    return train_dataset, val_dataset, test_dataset

def squeeze_excite_block(tensor, filters=2048):
    
    ## SE block code adapted from https://github.com/titu1994/keras-squeeze-excite-network
    
    init = tensor
    
    se = GlobalAveragePooling2D()(init)
    se = Reshape((1, filters))(se)
    se = Dense(filters // 16, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)
    se = multiply([init, se])
    
    return se
    
def model(IMG_SIZE):
    
    ## creating model for transfer learning
    
    input_tensor = tf.keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    
    x = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(include_top=False, weights='imagenet',
                                                       input_tensor=input_tensor, input_shape=(IMG_SIZE, IMG_SIZE, 3),
                                                       classes=10, pooling='None')
    x.trainable = False

    
    x = squeeze_excite_block(x.output, 1536)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.3)(x)
    x = Dense(1024, activation='relu', kernel_initializer='glorot_uniform')(x)
    x = Dropout(0.3)(x)
    x = Dense(512, activation='relu', kernel_initializer='glorot_uniform')(x)
    x = Dropout(0.3)(x)
    x = Dense(10, activation='softmax', kernel_initializer='glorot_uniform')(x)
    
    model = Model(inputs=input_tensor, outputs=x)
        
    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                 optimizer=Adamax(learning_rate=LEARNING_RATE),
                 metrics=[CategoricalAccuracy()])
    
    model.summary()
    
    return model

def save_model(model, model_name):
    
    model_directory = 'models'
    model_name = model_name + '.h5'
    if not os.path.exists(model_directory):
        os.makedirs(model_directory)

    model.save(f"{model_directory}/{model_name}")
    
    print("Saved model to disk")

def export_predictions(predictions, file_name):
    
    onlyfiles = [f for f in listdir(file_dir + '//test') if isfile(join(file_dir + '//test', f))]
    df = pd.DataFrame([int(x.replace('.png','')) for x in onlyfiles])
    df['Category'] = pred.argmax(axis=1)
    df = df.sort_values(by=0)
    df.columns = ['Id','Category']
    df.to_csv(file_name + '.csv', index=False)
    
    print('Predictions exported!')

In [3]:
train_dataset, val_dataset, test_dataset = png_to_tfds(file_dir, BATCH_SIZE, IMG_SIZE, SEED, VAL_SPLIT, AUTOTUNE, 'inception_resnet_v2')
model = model(IMG_SIZE)

Found 5641 files belonging to 10 classes.
Using 4513 files for training.
Found 5641 files belonging to 10 classes.
Using 1128 files for validation.
Found 13000 files belonging to 1 classes.


Size of training dataset is:  27264
Size of validation dataset is:  6912
Size of testing dataset is:  13000


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                        

In [8]:
checkpoint_filepath = './checkpoint'
model.load_weights(checkpoint_filepath)

In [ ]:
checkpoint_filepath = './checkpoint'
checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_loss')
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
train_model = model.fit(train_dataset,
                       validation_data=val_dataset,
                       batch_size=BATCH_SIZE,
                       epochs=EPOCHS,
                       callbacks=[checkpoint_callback, callback])

In [13]:
pred = model.predict(test_dataset, verbose=1)

13000/13000 [==============================] - 1035s 80ms/step


In [14]:
export_predictions(pred, 'predictions')

Predictions exported!


In [ ]:
#save_model(train_model, 'vgg16_attempt2')